# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [3]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [4]:
# Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [7]:
# Your code here; use a subquery. No join is necessary 
cur.execute('''SELECT customerNumber,
                      contactLastName
                      contactFirstName
               FROM customers
               WHERE customerNumber IN (SELECT customerNumber
                                        FROM orders
                                        WHERE orderDate = '2003-01-31')
       
       ;''')

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,customerNumber,contactFirstName
0,141,Freyre


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [13]:
# Your code here
cur.execute('''SELECT productName, COUNT(orderNumber) AS orderNumbers, SUM(quantityOrdered) AS TotalUnitsSold
               FROM products
               JOIN orderdetails
               USING(productCode)
               GROUP BY ProductName
               ORDER BY TotalUnitsSold DESC
       ;''')

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(df.shape)
df.tail()

(109, 3)


,productName,orderNumbers,TotalUnitsSold
104,1999 Indy 500 Monte Carlo SS,25,855
105,1911 Ford Town Car,25,832
106,1936 Mercedes Benz 500k Roadster,25,824
107,1970 Chevy Chevelle SS 454,25,803
108,1957 Ford Thunderbird,24,767


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [17]:
# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
cur.execute('''SELECT productName, COUNT(DISTINCT(customerNumber))
               FROM products
               JOIN orderdetails
               USING(productCode)
               JOIN orders
               USING (orderNumber)
               GROUP BY productName
       ;''')

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(df.shape)
df.tail()

(109, 2)


,productName,COUNT(DISTINCT(customerNumber))
104,The Mayflower,22
105,The Queen Mary,24
106,The Schooner Bluenose,23
107,The Titanic,22
108,The USS Constitution Ship,23


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [19]:
# Your code here
cur.execute('''SELECT employeeNumber, firstName, lastName, o.city, officeCode
               FROM employees e
               JOIN offices o
               USING (officeCode)
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders
               USING (customerNumber)
               JOIN orderdetails
               WHERE productCode IN (SELECT productCode
                                     FROM products
                                     JOIN orderdetails
                                     USING (productCode)
                                     JOIN orders
                                     USING (orderNumber)
                                     GROUP BY productCode
                                     HAVING COUNT(DISTINCT customerNumber <20)
               )
               
       ;''')

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(df.shape)
df.tail()

(976696, 5)


,employeeNumber,firstName,lastName,city,officeCode
976691,1702,Martin,Gerard,Paris,4
976692,1702,Martin,Gerard,Paris,4
976693,1702,Martin,Gerard,Paris,4
976694,1702,Martin,Gerard,Paris,4
976695,1702,Martin,Gerard,Paris,4


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [21]:
# Your code here
cur.execute('''SELECT employeeNumber, firstName, lastName, COUNT(customerNumber) AS numCustomers
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               GROUP BY employeeNumber, firstName, lastName
               HAVING AVG(creditLimit) > 15000               
       ;''')

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(df.shape)
df.tail()

(15, 4)


,employeeNumber,firstName,lastName,numCustomers
10,1504,Barry,Jones,9
11,1611,Andy,Fixter,5
12,1612,Peter,Marsh,5
13,1621,Mami,Nishi,5
14,1702,Martin,Gerard,6


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!